## Introduction to Databases with Python
----------------------------------------------------------------------------------------

![sqlitepython](http://www.sqlitetutorial.net/wp-content/uploads/2016/01/SQLite-Python.jpg)

# WTF is a database, dude?
----------------------------------------------------------------------------------------
* A collection of data, can scale better than simple(flat) file.
* Usually designed to model real world(OOP someone?).
* Offers some kind of query language for CRUD operations.
* Most popular are the relational databases.

# Now WTF is a relational database?
-----------------------------------------------------------------------------------------

> This(relational) model organizes data into one or more tables (or "relations") of columns and rows, with a unique key identifying each row. Rows are also called records or tuples.[2] Columns are also called attributes. Generally, each table/relation represents one "entity type" (such as customer or product). The rows represent instances of that type of entity (such as "Lee" or "chair") and the columns representing values attributed to that instance (such as address or price).
                                                                                    Wikipedia
                                                                                    
* Relationships between entities can solve real world problems(e.g. John(**entity-customer**) buys 1 Big Mac(**entity-product**).)
* The query language is **SQL**- many different flavors but the underlying concepts remain the same.

# Why it is useful?
-----------------------------------------------------------------------------------------
![](https://www.lee.k12.nc.us/cms/lib/NC01001912/Centricity/Domain/2133/suggestions.jpg)

## Just because it is
------------------------------------------------------------------------------------------

* Separation of program logic(code) and data.
* Same data can be reused in many different applications.
* More sophisticated and flexible than flat file. What do you think if 20+ people are opening and saving the same file at the same time? What if somebody forgets to close it? 

Remember:
* Files corrupt easily and generally are not designed for concurrent multi-user read-write. 
* Also when you open them you load the whole thing in your RAM memory. Not the case with databases.
* How do you query a file? Errr....let's just use SQLite.

# What is SQLite 3?
----------------------------------------------------------------------------------------------

* Relational database engine. Comes with your Python installation.
* Your whole database is kept in a single file.
* Suitable for small to medium scale projects.

![signmeup](http://greatbridgewrestlingclub.com/wp-content/uploads/2014/08/sign-me-up.jpg)

# Using SQLite with Python
------------------------------------------------------------------------------

```python
import sqlite3

conn = sqlite3.connect('progress.db')
c = conn.cursor()
```

Now go check the folder your program is in. There is another file there. Takeout: if we connect to a database that does not yet exist, SQLite creates it.

Also you can create a in-memory database using the following connection string:
```
c1 = sqlite3.connect("file::memory:?cache=shared")
```
By default SQLite does not offer any password protection so whoever has the db file has your data. Not perfect, but many solutions. You can research the official encryption extension [here](http://www.hwaci.com/sw/sqlite/see.html).

# I want to create and update a table
----------------------------------------------------------------------------------------------------------------

All the transactions are executed using the cursor. Let's see the official example from the docs:
```python
# Create table
c.execute('''CREATE TABLE IF NOT EXISTS stocks
             (date text, trans text, symbol text, qty real, price real)''')

# Insert a row of data
c.execute("INSERT INTO stocks VALUES ('2006-01-05','BUY','RHAT',100,35.14)")

# Save (commit) the changes
conn.commit()

# We can also close the connection if we are done with it.
# Just be sure any changes have been committed or they will be lost.
conn.close()
```
See how after transactions are executed, they have to be commited to take effect. Also like we did with files we have to close the connection when we are done.

# Onto the cool stuff now
----------------------------------------------------------------------------------------
We can execute database queries in functions and with variables from our program. 
```python
def insert_row(date,trans,symbol,qty,price):
    c.execute("INSERT INTO stocks (date,trans,symbol,qty,price) VALUES (?,?,?,?,?)",(date,trans,symbol,qty,price))
    conn.commit()
    
insert_row('2011-01-05','SELL','APPL',10000,11.12)
```
In order to make the execute more Python 3 looking we can do:
```python
c.execute("INSERT INTO stocks (date,trans,symbol,qty,price) VALUES ('{}','{}','{}',{},{})".format(date,trans,symbol,qty,price))
```
Note that setting database entries using variables this way can break the whole database. 
We can select entres from the database:
```python
c.execute('SELECT * from stocks')
for row in c.fetchall()
    print(row)
```
Try to execute the following query by yourself: SELECT * FROM stocks WHERE trans='BUY'
Then try SELECTing just one column from the table.

```python
c.execute('SELECT * FROM stocks where trans=?',('BUY',))
```

# Context managers a.k.a. with statement and SQLite
-----------------------------------------------------------------------

```python
def insert_row(date,trans,symbol,qty,price):
    with conn:
        c.execute("INSERT INTO stocks (date,trans,symbol,qty,price) VALUES (?,?,?,?,?)",(date,trans,symbol,qty,price))
```

# UPDATE and DELETE - point of no return
-----------------------------------------------------------------------------

```python
c.execute('UPDATE stocks SET price=1 WHERE price=11.12')
conn.commit()
```

```python
c.execute('DELETE FROM stocks WHERE price=1')
conn.commit()
```

# DUMPing database to a file
-----------------------------------------------------------------------------
```python
with open('dump.sql','w') as f:
    for line in conn.iterdump():
        f.write("{}\n".format(line))
```

## SQLAlchemy
----------------------------------------------------------------------------------------------
![](https://www.fullstackpython.com/img/logos/sqlalchemy.jpg)

# SQLAlchemy is an ORM
--------------------------------------------------------------------

>Object-Relational Mapping (ORM) is a technique that lets you query and manipulate data from a database using an object-oriented paradigm. When talking about ORM, most people are referring to a library that implements the Object-Relational Mapping technique, hence the phrase "an ORM".

|Pros of ORMs:                                         |                     Cons of ORMs:                            |
|----------------------------------------------------- | -------------------------------------------------------------|
|Feel more natural if you know Python                  |    Steep learning curve, not easy at all                     |
|A lot of database handling stuff is done automatically|    It is a pain to set up right                              |
|Abstracts the database, so not database agnostic      |    You have no direct access to the database,only abstraction|
|You can use OOP concepts(e.g inheritance)             |    SQL queries can be better fine tuned for performance      |

![](https://lh4.googleusercontent.com/Xzlor68rkHmdKNlSyfoVEokbZsbeCrsiJyqlsDl99Rppe_3UvDM-CGoaj4vboovuCd4Br1p8DFrJBJSvTdmJ=w1920-h941)

# How to get started
---------------------------------------------------------------------------------------------------

```python
pip install SQLAlchemy
```

Import some modules from the library in your python script to test if it is installed right:

```python
from sqlalchemy import Table, Column, Integer, String
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
```

![](https://static1.squarespace.com/static/5286828ae4b08b83ea176e95/t/56d755e486db43c1b33edca5/1456952807640/)

# Next steps
-----------------------------------------------------------------
Create a database engine connection:
```python
engine = create_engine('sqlite:///employees.db')  #Try setting echo=True option
```
Then create our Base class from which our classess will inherit:
```python
Base = declarative_base()
```
And open a new database session(similarly to the cursor with the plain sqlite library):
```python
Session = sessionmaker(bind=engine)
session = Session()
```

Now we are game on!

# Let's experience some ORM goodness
-----------------------------------------------------------------------------
Create an Employee class like this:
```python
class Employee(Base):
    __tablename__  = 'Employees'
    
    id = Column('id', Integer, primary_key=True)
    first = Column('first', String)
    last = Column('last', String)
    pay = Column('pay', Integer)
    
    def __init__(self, id, first, last, pay):
        self.id = id
        self.first = first
        self.last = last
        self.pay = pay
```
Don't worry I will explain it :)

Now create all tables inheriting from Base(we have only Employees for now):
```python
Base.metadata.create_all(bind=engine)
```

# See what I did there?
----------------------------------------------------------------------------------------------------

Now when we create an employee, a record for it gets added to the table Employees in our database:

```python
john = Employee(0,"Johny","Doe",50000)
session.add(john)
session.commit()
```
Or if we want to add multiple objects at once instead of add we use:
```python
mary = Employee(1,"Mary","Johnson",24000)
kate = Employee(2,"Kate","Jackson",13000)
session.bulk_save_objects([john,mary,kate])
```
We can also execute SQL queries the ORM way. This is equivalent to __UPDATE Employee SET pay = pay +1 WHERE first='Johny'__:
```python
session.query(Employee).filter_by(first="Johny").update({Employee.pay: Employee.pay + 1})
session.commit()
```

And this is equivalent to __SELECT first FROM Employee__:
```python
for employee in session.query(Employee).all():
    print(employee.first)
```

# Loading data from file into a database
---------------------------------------------------------------------------------------------

Get the temperature deviation dataset from [here](http://tiny.cc/tempdev).

Install the pandas library.

Then load the csv file into the database like this:
```python
from sqlalchemy import create_engine
import pandas as pd

engine = create_engine('sqlite:///temperature.db')
content = pd.read_csv('TempVariance1880-2017.csv',sep=';')
content.to_sql(name='temperatures', con=engine)

```

# Plotting/Charting
----------------------------------------------------------------------------------------------

Let's get back to our Employee class and database. What if we want to create a barchart with the names of the employees on the horizontal axis and their corresponding salaries on the vertical.

So we need something like this

```python
x = [john,mary,kate]
y = [50000,24000,13000]
```
How do query the database?

```python
for employee in session.query(Employee).all():
    x.append(employee.first)
    y.append(employee.pay)
```


![](https://matplotlib.org/_static/logo2.svg)
![](http://wiki.openhatch.org/images/d/d8/Matplotlib_gallery.png)

![](http://matplotlib.1069221.n5.nabble.com/file/n39657/image.png)

# Add the following to your code to get a nice chart
----------------------------------------------------------------------------
```python
import matplotlib.pyplot as plt

plt.bar(x,y)
plt.show()
```

# Once we plotted from database, let's plot from csv
------------------------------------------------------------------------------------------------------

```python
import pandas as pd
import matplotlib.pyplot as plt

temperatures = pd.read_csv('TempVariance1880-2017.csv',sep=';')
print(temperatures)
x = temperatures['Year']
y = temperatures['Value']
plt.bar(x, y)
plt.show()
```


# Some other things you should know
-----------------------------------------------------------------------------------

You can change the style of the chart like this:
```python
plt.style.use("ggplot")
```
I like the fivethirtyeight style.

Also you can add title and x and y labels to the chart:
```python
plt.title("Global Land and Ocean Temperature Anomalies, June",fontsize=15)
plt.xlabel("year")
plt.ylabel("degrees F +/- from average")
```

Simiarly to showing the chart you can save it to a file
```python
plt.savefig('sampleFileName.png')
```

# We have bar chart what about a line chart?
-----------------------------------------------------------------
```python
import matplotlib.pyplot as plt

x = [1,2,3]
y = [5,7,4]

x2 = [1,2,3]
y2 = [10,14,12]

plt.plot(x, y, label='First Line')
plt.plot(x2, y2, label='Second Line')

plt.xlabel('Plot Number')
plt.ylabel('Important var')
plt.title('Interesting Graph\nCheck it out')
plt.legend()
plt.show()
```